# Part 6. Spectrum Sharing

**Objectives:** 
* Understand the basics of spectrum sharing and perform an experiment on spectrum sharing with multiple teams
* Use a collaboration channel to improve the efficiency of finding a good spectrum sharing solution

**Required Materials**
* Computer speakers
* Python audio libraries noted below
* Printouts of the Morse Code chart (file: morse.pdf) for each lab participant. 

## Loading the Necessary Libraries


In [ ]:
# These are the minimum sound libraries required. 

import scipy.io.wavfile as wavfile
import sounddevice as sd
import pyaudio

In [ ]:
# Library for visualizing the sound recordings

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Helper libraries

import numpy as np
import time


In [ ]:
# The code in this block loads some elements for adding interactive 
# widgets to the code

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import IPython.display as display



In [ ]:
# This notebook uses a couple of new functions from NumPy to transform a signal into a frequency
# representation

import numpy.fft

## Spectrum Sharing

The radio frequency band can be divided among users in different ways. In cellular (mobile phone) communications, the carriers (like T-Mobile or Verizon) buy exclusive rights to use particular sets of frequencies in different regions.

In WiFi, people setting up wireless access points try to choice different frequencies than their neighbors in order to avoid the signals colliding.


Future cellular communication systems may use dynamic spectrum sharing techniques, in which intelligent radios automatically determine a frequency band to use. The radios may use criteria, such as whether a particular frequency band is in use, or may exchange credit with other radios to buy exclusive access to a frequency band. 

In this segment of the lab, we will explore dynamic spectrum sharing, where you and your teammates are the intelligence behind the radios.  Instead of using the RF spectrum, we will experiment with sharing the audible spectrum.

**Activity 6.1:  Volume Normalization**

Lab organizer -- divide the students in up to 4 teams. Each team needs at least 2 people and 2 computers. Determine a unique numbering (from 1 to 4 if there are 4 teams) for the teams. For instance, find out the earliest or latest time that anyone got up in each team, and then use the order of those times to number the teams.

In this exercise, one computer with a speaker will act as the transmitter, and one computer with a microphone will act as the receiver. Ideally, the transmitters should be in in a rough line across from the receivers in a similar line. 

At each team's transmitter and receiver, have the teams replace the 0 in the cell below with their team number and then run the cell:

In [ ]:
team_number = 0 # Change the 0 to your team's number

The way we will share the spectrum is by each team using one of the following tones: 400 Hz, 600 Hz, 800 Hz, and 1000 Hz. Teams will simultaneously transmit a message that contains a randomly chosen animal using Morse Code. The first step will be to adjust the output volume on the transmitters to similar levels. Use the following procedure:

1. Have Team 1 run the code below and adjust the volume to a level at which the tone is clearly heard from the receivers.
2. Now have Team 1 run the code. As soon as the tone finishes, have Team 2 run the code and adjust the volume to try to match Team 1's transmission volume. Repeat as necessary.
3. Repeat the procedure in step 2 for Team 1 and each of Team 3 and Team 4. 
4. Have each team run the code below in sequence and make any final volume adjustments.

In [ ]:
# Play tone for adjusting volumes of different transmitters

f_c = 700  # 700 Hz tone in the middle of the four transmission frequencies

rate=44100

# Create the sinusoid:
total_duration=2
t=np.linspace(0, total_duration, int(total_duration*rate) )
carrier = np.sin(2 * np.pi * f_c * t)


sd.play(carrier)

### Spectrum Sharing with Fixed Assignments 1


**Activity 6.2: Simultaneous Transmission Fixed Frequencies**

Once the volumes are adjusted, we will begin experimenting with our spectrum sharing experiments. As a first experiment, we will test the ability of our ears to discriminate among signals at different frequencies. Choose one team member to operate the transmitter; this team member is not allowed to guess the transmitted message. The other team members should stand by the receiver computers (to separate them from the transmitter) and try to work as human receivers. Each team member should have a printout of the Morse Code table to help with this.

At a signal from the lab organizer, have teams simultaneously run the cell below, according to the following plan:

1. Teams 1 and 3 run the block below simultaneously.
2. Teams 2 and 4 run the block below simultaneously.
3. Teams 1, 2, and 4 run the block below simultaneously.
4. Teams 1, 3, and 4 run  the block below simultaneously.
5. All teams run the block below simultaneously.

**Whenever your team transmits, record the message guessed by the human receivers and the actual message transmitted.**

In [ ]:
def transmit_freq(freq_index):
    '''
    Transmit a randomly chosen message at one of several
    pre-selected frequencies.

    Parameters
        freq_index = used to choose from preset group of frequencies
        
    Returns
        N/A


    Note that when you run  this block, the function is defined, but
    it will not record any audio until you call this function in a later block

    John M. Shea
    6/1/2021
    '''

    dit_duration=0.20
    all_frequencies = [400, 600, 800, 1000]

    if freq_index not in [1,2,3,4]:
        print("Before running this cell, set the team_number above and run that cell (shift-Enter)!")
        raise NameError("Wrong team_number")

    f_c = all_frequencies[freq_index-1]

    codex = {
        'cat' : np.array([[1,1,1,0,1,0,1,1,1,0,1,0,0,0,1,0,1,1,1,0,0,0,1,1,1]]),
        'dog' : np.array([[1,1,1,0,1,0,1,0,0,0,1,1,1,0,1,1,1,0,1,1,1,0,0,0,1,1,1,0,1,1,1,0,1]]),
        'fox' : np.array([[1,0,1,0,1,1,1,0,1,0,0,0,1,1,1,0,1,1,1,0,1,1,1,0,0,0,1,1,1,0,1,0,1,0,1,1,1]]),
        'bear': np.array([[1,1,1,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,1,1,1,0,0,0,1,0,1,1,1,0,1]])
    }
    animals=list(codex.keys())
    animal=animals[ np.random.randint(len(animals))]

    info=codex[animal]
    rate=44100

    # Create the sinusoid:
    total_duration=np.shape(info)[1]*dit_duration
    t=np.linspace(0, total_duration, int(total_duration*rate) )
    carrier = np.sin(2 * np.pi * f_c * t)

    # Each bit needs to last bit_duration*rate samples:
    repeated_bits = np.repeat(info.T, dit_duration * rate, axis=1)

    # Make the repeated bits into one long signal vector
    info_signal = np.reshape(repeated_bits, -1)

    output_signal= info_signal*carrier


    time.sleep(1)
    sd.play(output_signal)
    print('Which of these animals was transmitted?')
    print('1 - cat')
    print('2 - dog')
    print('3 - fox')
    print('4 - bear')
    print()
    print('Type the number (or type r to repeat) and press Enter', end='')

    guess=input()

    if animals[int(guess)-1] == animal:
        print(f'Correct! The animal was {animal}')
    else:
        print(f'Incorrect. The animal was {animal}')




In [ ]:
transmit_freq(team_number)

### Spectrum Sharing with Fixed Assignments 2


**Activity 6.3: Using a Tuned Receiver**

Now let's use a receiver that uses signal processing to filter out the other signals and remove the carrier from the desired signal. You do not need to understand how this receiver works, but a brief explanation follows: This is a *tuned* receiver that multiplies the incoming signal with a sinusoid at the same frequency. This will result in a copy of the signal with the frequency tone removed and a copy of the signal at twice the frequency. We then use a low-pass filter to remove the double frequency component and any signals at other frequencies.

Each team should open this notebook on the receiver computer and run all the cells at the top to load the libraries.

Teams should take turns doing the following:

The transmitter is going to run the `transmit_freq()` code, as above. 

The receiver should run the `receive()` function below. Here the `freq_index` parameter should be set to the `team_number`.

**Note that both functions should be run at approximately the same time so that the receiver can capture the entire transmitted signal.**

*Sketch what the output of the tuned receiver looks like.*

In [ ]:
def receive(freq_index, duration = 12, rate = 44100):
    # Record audio from the system standard audio device, 
    # plot the output, and save the data as a WAV file.
    #
    # Audio is captured as soon as the function is run.
    #
    # Parameters
    #     freq_index = used to choose from preset group of frequencies
    #     duration = length of audio recording (defaults to 10s) 
    #     rate = sampling rate (44100 is widely compatible)
    #
    # Returns
    #     rate = sampling rate (to have same outputs as wavfile.read() )
    #     data = data in ndarray format
    #
    # 
    # Note that when you run  this block, the function is defined, but
    # it will not record any audio until you call this function in a later block
    #
    # John M. Shea
    # 6/1/2021

    all_frequencies = [400, 600, 800, 1000]
    f_c = all_frequencies[freq_index-1]


    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=rate,
                    input=True,
                    frames_per_buffer=CHUNK)
    display.clear_output()
    print("* recording")

    frames = []

    for i in range(0, int(rate / CHUNK * duration)):
        data = stream.read(CHUNK)
        frames.append(data)


    print("* done recording")

    stream.stop_stream()
    stream.close()
    p.terminate()

    time=np.linspace(0, duration, int(rate/CHUNK*duration)*CHUNK)
    #data=np.frombuffer(frames[0], dtype=np.int16)
    #for frame in range(1,len(frames)):
    #    data=np.hstack((data, np.frombuffer(frames[frame], dtype=np.int16)))
    data = np.frombuffer(b''.join(frames), dtype=np.int16)
    
    plt.plot(time, data)
    plt.xlabel('Time (s)')
    plt.ylabel('Relative sound pressure')
    plt.title('Captured audio signal')

    fft1 = np.fft.fft(data)
    freqs1 = np.fft.fftfreq(len(data), 1/rate)
    plt.figure()
    plt.plot(freqs1, abs(fft1) )
    plt.title('Power distribution across frequencies')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power density')


    plt.xlim(-1500, 1500)

    # Now do some signal processing to remove the carrier wave
    csin=np.exp(1j*2*np.pi*f_c*time)
    mixed=csin*data

    # And now filter out any other signals
    order = 10 
    cutoff = 100
    sos= butter(order, cutoff, btype='low',fs=rate,  output='sos')
    FilterData = sosfilt(sos, mixed/mixed.max())
    
    plt.figure()
    plt.plot(time, np.abs(FilterData) )
    plt.xlabel('Time (s)')
    plt.ylabel('Captured power')
    plt.title(f'Power at frequency {f_c} Hz')
    
    
    return rate, data

In [ ]:
# The transmitter runs only this one:

transmit_freq(team_number)

In [ ]:
# The receiver runs only this one:
receive(team_number)

### Dynamic Spectrum Sharing Experiment 1: Three Teams

**Activity 6.4: Dynamic Spectrum Sharing with Three Teams, Round 1**

We are now ready to carry out our first dynamic spectrum sharing experiment. If you have 4 teams, choose one to sit out the first time this experiment is run.  The remaining 3 teams will try to find a set of frequencies to use that allow each team to receive the signal sent by their team's transmitter.  (The team that sits out this round will participate in a second run of this experiment.)

**IMPORTANT:** Teams are not allowed *any* communication with the other teams during this experiment. We will allow limited communication in later experiments.

Each experiment consists of a number of rounds. Repeat the following until all teams are able to accurately determine the message sent by their team's transmitter:

1. Each team chooses one member to operate their team's transmitter. All other team members will work at the receiver.
1. Each team picks a frequency index from 1 to 4. Teams are not allowed to use their team number in the first 2 rounds. If a team was able to recover their message in the previous round, they should generally reuse the same frequency index as in the previous round. Teams that experienced interference and were not able to recover the transmitted message may decide to switch or not switch because if two teams that were using the same frequency both switch, then they both may switch to the same frequency and cause interference again.
1. At a signal from the lab leader, the participating teams will run their transmitters and receivers.
1. At the receivers, the team members will study the plots and try to decode the Morse-coded words. The members will then tell the team member that is working the transmitter what they think the message was. 
1. At each transmitter, the team member will use the message specified by the receiver team to answer the question about which message was transmitted. The transmitter will then tell the receiver team whether the message was correctly decoded or not.
1. If all teams have recovered their messages correctly, the experiment is complete. Otherwise, the next round begins at step 2 above.



In [ ]:
# Change chosen_freq to the desired channel_index

chosen_freq=0

In [ ]:
# The transmitter runs only this one:

transmit_freq(chosen_freq)

In [ ]:
# The receiver runs only this one:
receive(chosen_freq)

**Questions:** What is the final list of frequencies used by each team? How many rounds were required?

### Dynamic Spectrum Sharing Experiment 2: Three Teams

**Activity 6.5: Dynamic Spectrum Sharing with Three Teams, Round 1**

If a team had to sit out in Dynamic Spectrum Sharing Experiment 1, repeat the experiment with three different teams. Teams are not allowed to use their team_number or their frequency index from experiment 1 in the first 2 rounds.

**Questions:** What is the final list of frequencies used by each team? How many rounds were required?

### Dynamic Spectrum Sharing Experiment 3: Four Team, Sequential


**Activity 6.6: Dynamic Spectrum Sharing with Four Teams, Sequential**

In practice, it is unlikely that all of the users of a system would start accessing the channel at the same time. In this experiment, teams will begin transmitting one by one. 

1. Run the cell below to choose a random order for the teams to begin transmitting. The first team listed will transmit beginning in the first round and every round thereafter, the second team listed will transmit beginning in the second round and every round thereafter, etc.
2. When a team is not transmitting, that team can still run the receive() function in order to see which channels have power in them. 
3. When a team begins transmitting, it should use a transmission frequency that it believes is not being used by any other team. This can determine by listening to the tones being sent or by looking at the plot of power distribution by frequency. Once a team transmits on a frequency, it should continue to use that same frequency provided that they were able to recover the transmitted signal. The team member at the transmitter should tell the team members at the receiver which frequency it is transmitting on. 
3. During a round, the team members at the receiver will tell the team member at the transmitter which animal (i.e., message) they have decoded. The team member at the transmitter should then inform the team members at the receiver whether they were correct or not. If the team chose a frequency that was already used by another team and were not able to recover their signal because of interference, then that team should choose another frequency in the next round.
4. Ideally, every team should have a unique frequency after 4 rounds, and every team should be able to recover their message in round 4. If not, continue with additional rounds until all teams can recover their message.



In [ ]:
# The lab leader should run  this to determine the order in which teams should begin transmission

teams=[1,2,3,4]
np.random.shuffle(teams)
print(teams)

In [ ]:
# Change chosen_freq to the desired channel_index

chosen_freq=0

In [ ]:
# The transmitter runs only this one:

transmit_freq(chosen_freq)

In [ ]:
# The receiver runs only this one:
receive(chosen_freq)

**Questions:**  What is the final list of frequencies used by each team?  How many rounds were required? If it was more than 4, try this experiment again and use the plots of power distribution across frequency to try to determine which frequencies are being used in previous rounds. Be sure to select a transmission frequency that is not already being used by another team.

### Dynamic Spectrum Sharing Experiment 4: Collaborative Spectrum Sharing


**Activity 6.7: Collaborative Spectrum Sharing with Sufficient Channels**

In collaborative spectrum sharing, users a channel share information about their use of the available frequencies to enable the teams to use the spectrum efficiently. This information is sometimes exchanged over a special *collaboration channel*. In this experiment, team members will use voice communication for the collaboration channel. 

1. The lab leader should run the cell directly below to choose the team order. This is the order in which teams will announce their planned channel usage.
2. At the beginning of each round, teams will take turns announcing their planned frequency use. The order in which teams announce their planned frequency will be specified by the lab leader (according to the random order selected in step 1).
1. At a signal from the lab leader, the participating teams will run their transmitters and receivers.
1. At the receivers, the team members will study the plots and try to decode the Morse-coded words. The members will then tell the team member that is working the transmitter what they think the message was. 
1. At each transmitter, the team member will use the message specified by the receiver team to answer the question about which message was transmitted. The transmitter will then tell the receiver team whether the message was correctly decoded or not.
1. If all teams have recovered their messages correctly, the experiment is complete. Otherwise, the next round begins at step 2 above.




In [ ]:
# The lab leader should run this to determine the order in which teams should 
# announce their planned transmission frequency

teams=[1,2,3,4]
np.random.shuffle(teams)
print(teams)

In [ ]:
# Change chosen_freq to the desired channel_index

chosen_freq=0

In [ ]:
# The transmitter runs only this one:

transmit_freq(chosen_freq)  

In [ ]:
# The receiver runs only this one:

receive(chosen_freq)

**Questions:** How many rounds were required? If more than 1 round was required, conduct the experiment again. How does this approach compares to the approach in Experiment 3 in terms of using the available frequencies efficiently?



**Activity 6.7: Collaborative Spectrum Sharing with Insufficient Channels**

Repeat experiment 4, but only use the frequencies 1, 2, and 3. Teams do not all have to transmit in a round, but the goal is for each team to deliver 3 messages over a series of 4 rounds.

1. The lab leader should run the cell directly below to choose the team order. This is the order in which teams will announce which frequency they plan to use or whether they will not transmit.
2. At the beginning of each round, teams will take turns announcing their planned frequency use (or if they will not transmit). The order in which teams announce their planned frequency will be specified by the lab leader (according to the random order selected in step 1).
1. At a signal from the lab leader, the participating teams will run their transmitters and receivers.
1. At the receivers, the team members will study the plots and try to decode the Morse-coded words. The members will then tell the team member that is working the transmitter what they think the message was. 
1. At each transmitter, the team member will use the message specified by the receiver team to answer the question about which message was transmitted. The transmitter will then tell the receiver team whether the message was correctly decoded or not.
1. If this is the fourth  round, then the experiment is complete. Otherwise, start another round by going to step 2.




In [ ]:
# The lab leader should run this to determine the order in which teams should announce
# their planned transmission frequency (or whether they will remain quiet)

teams=[1,2,3,4]
np.random.shuffle(teams)
print(teams)

In [ ]:
# Change chosen_freq to the desired channel_index

chosen_freq=0

In [ ]:
# The transmitter runs only this one:

transmit_freq(chosen_freq)  

In [ ]:
# The receiver runs only this one:

receive(chosen_freq)

**Questions:** 

* What rounds did each team transmit in?
* How many messages were delivered by each team? 

*If each team did not deliver 3 messages, have the teams discuss how they can achieve the desired goal. Then run the experiment again.*

**Activity 6.8: Collaborative Spectrum Sharing with Insufficient Channels**


Repeat experiment 5, using only frequency indices 1, 2, and 3. Tell teams that they do not all have to transmit, but each team has the goal of getting their own message through **4 times** in 4 rounds.

1. The lab leader should run the cell directly below to choose the team order. This is the order in which teams will announce which frequency they plan to use or whether they will not transmit.
2. At the beginning of each round, teams will take turns announcing their planned frequency use (or if they will not transmit). The order in which teams announce their planned frequency will be specified by the lab leader (according to the random order selected in step 1).
1. At a signal from the lab leader, the participating teams will run their transmitters and receivers.
1. At the receivers, the team members will study the plots and try to decode the Morse-coded words. The members will then tell the team member that is working the transmitter what they think the message was. 
1. At each transmitter, the team member will use the message specified by the receiver team to answer the question about which message was transmitted. The transmitter will then tell the receiver team whether the message was correctly decoded or not.
1. If this is the fourth  round, then the experiment is complete. Otherwise, start another round by going to step 2.




In [ ]:
# The lab leader should run this to determine the order in which teams should announce
# their planned transmission frequency (or whether they will remain quiet)

teams=[1,2,3,4]
np.random.shuffle(teams)
print(teams)

In [ ]:
# Change chosen_freq to the desired channel_index

chosen_freq=0

In [ ]:
# The transmitter runs only this one:

transmit_freq(chosen_freq)  

In [ ]:
# The receiver runs only this one:

receive(chosen_freq)

**Questions:** 

* What rounds did each team transmit in?
* How many messages were delivered by each team? 
* What was the total number of messages delivered?
* How did the total number of messages delivered compare to the last experiment?
* Which of Activity 6.7 and Activity 6.8 are most like a real system? 
* How could real systems be incentivized to behave like those in Activity 6.7 instead of those in Activity 6.8?


## Review

* Most current communication systems use *fixed channel assignment* in which each system is assigned a particular frequency band (in some area) that no one else is allowed to use. 
* Fixed channel assignments are wasteful when the assigned user doesn't use the band continuously.
* In *dynamic spectrum access*, users choose channels and try to avoid disrupting each other's communications.
* In *collaborative spectrum sharing*, users exchange information to help do a better job at dynamic spectrum access.
* Methods are needed to incentivize cooperation when there are fewer channels available then there is systems that want to use those channels.